## What's a WebHook?
A webhook is a way for an application or service to provide real-time notifications to another application or service by sending HTTPS requests to a specific URL (known as the webhook endpoint) when a specific event or trigger occurs.

For Example: we want to keep track of a delete protected service and if something occurs where it gets accidentally deleted, we can trigger a script to be run to reinstate the service.

![webhookvsapi](https://images.ctfassets.net/lzny33ho1g45/5UrIAftDqQqfdEteoHgVQI/4ada8fc34fbcf012e730d0953dc8917f/webhook-vs-apis-differences.png?w=700)

# How is it useful to admins?
  
  By leveraging webhooks, an ArcGIS Online admin can streamline workflows, automate processes, and enhance integration with external systems. The specific use cases and implementations will depend on the requirements and objectives of your organization.

> ## WebHookServiceManager
> This manager is used for working with feature service-specific events in the ArcGIS API for Python. It allows owners and administrators to wire feature service events to a FeatureLayerCollection object. It is primarily focused on managing webhooks related to feature service events, such as when features are added, updated, or deleted. 
  
    
> ## WebHookManager
> This manager enables you to receive notifications for events associated with items, groups, and users within your ArcGIS Organization environment. It allows you to configure webhooks for various events, including item creation, deletion, modification, user authentication events, and more.

#### Establish a connection

In [15]:
from arcgis.gis import GIS
from arcgis.features.managers import WebHookEvents

gis = GIS(profile="your_online_profile")

## Create a Feature Layer Collection from a File Geodatabase

In [3]:
# Create a feature layer collection
fp = "C:\ipython_workfolder\Data\FileGeoDataBase\webhook_data.zip"
item = gis.content.add(
            {"type": "File Geodatabase", "tags": "UC2023", "title": "UC_WebHook_Demo"},
            data=fp,
        )
pitem = item.publish()

flc = pitem.layers[0].container
flc

<FeatureLayerCollection url:"https://services7.arcgis.com/JEwYeAy2cc8qOe3o/arcgis/rest/services/UC_WebHook_Demo/FeatureServer">

## Update properties of the service

In [4]:
# Configure behavior and permissions related to data access, editing, and tracking
properties_dict = {
            "hasStaticData": False,
            "capabilities": "Query,Editing,Create,Update,Delete,ChangeTracking",
            "editorTrackingInfo": {
                "allowAnonymousToDelete": True,
                "allowAnonymousToUpdate": True,
                "allowOthersToDelete": True,
                "allowOthersToQuery": True,
                "allowOthersToUpdate": True,
                "enableEditorTracking": False,
                "enableOwnershipAccessControl": False,
            },
        }
flc.manager.update_definition(properties_dict)

{'success': True}

## Get the WebHook Manager instance

In [5]:
# Get the WebHookServiceManager off of the feature layer collection manager
webhook_manager = flc.manager.webhook_manager
webhook_manager

< WebHookServiceManager @ https://services7.arcgis.com/JEwYeAy2cc8qOe3o/arcgis/rest/admin/services/UC_WebHook_Demo/FeatureServer/WebHooks >

## Create a WebHook

In [6]:
# Define where the webhook notification will be sent (MUST BE https)
hook_end_point_url = "https://webhook.site/fae5e5b9-b3a5-4dcf-bf26-25c78acbd11b"

In [7]:
# Create a webhook. Pass in a name, the target url, and set active
webhook = webhook_manager.create("UC2023_webhook", hook_end_point_url, active=True)

# Webhook properties from new webhook
webhook.properties

{
  "name": "UC2023_webhook",
  "owner": "naubry_geosaurus",
  "id": 139,
  "globalId": "f1e47061-5840-4311-96eb-db0100b64bed",
  "tenantId": 25830,
  "serviceId": 26841,
  "active": true,
  "hookUrl": "https://webhook.site/fae5e5b9-b3a5-4dcf-bf26-25c78acbd11b",
  "serviceUrl": "https://services7.arcgis.com/JEwYeAy2cc8qOe3o/arcgis/rest/services/UC_WebHook_Demo/FeatureServer",
  "signatureKey": "",
  "format": "json",
  "serverGen": 968777,
  "contentType": "application/x-www-form-urlencoded",
  "createdTime": 1689112203282,
  "lastUpdatedTime": 1689112203282,
  "changeTypes": [
    "*"
  ],
  "scheduleInfo": {
    "name": "",
    "state": "enabled",
    "recurrenceInfo": {
      "frequency": "second",
      "interval": 30
    }
  }
}

### Update a feature to see the webhook

In [8]:
cities = pitem.layers[0]

In [9]:
cities_fset = cities.query()
cities_fset.sdf.head()

,OBJECTID_1,Id,ADMIN_NAME,CITY_NAME,CNTRY_NAME,FIPS_CNTRY,GMI_ADMIN,LABEL_FLAG,ObjectID,POP,POP_CLASS,POP_RANK,PORT_ID,STATUS,GlobalID,SHAPE
0,1,0,Mato Grosso,Cuiaba,Brazil,BR,BRA-MGR,0,0,521934,"500,000 to 999,999",3,0,Provincial capital,9ce8080d-0391-4349-a201-a0496c27cb67,"{""x"": -6244244.6062, ""y"": -1760180.1805000007,..."
1,2,0,Kentucky,Frankfort,United States,US,USA-KEN,0,500,16315,"Less than 50,000",7,0,Provincial capital,68460c29-1db8-4bee-b6cc-495f560810a5,"{""x"": -9444234.6816, ""y"": 4607859.9877, ""spati..."
2,3,0,Tennessee,Nashville,United States,US,USA-TNN,0,501,530852,"500,000 to 999,999",3,0,Provincial capital,48045d62-dfd8-48d2-a152-543c4de039a1,"{""x"": -9664535.075, ""y"": 4320178.250500001, ""s..."
3,4,0,Distrito Federal,Brasilia,Brazil,BR,BRA-DFD,0,1,2207718,"1,000,000 to 4,999,999",2,0,National and provincial capital,d9d102a1-775a-41d2-ae77-fc3d52899dbb,"{""x"": -5331952.8794, ""y"": -1780660.5108999982,..."
4,5,0,Goias,Goiania,Brazil,BR,BRA-GOI,0,2,1171195,"1,000,000 to 4,999,999",2,0,Provincial capital,84fe805d-dad1-40d4-be66-d7640eeda9b9,"{""x"": -5483041.6975, ""y"": -1889069.6970999986,..."


In [10]:
cities_features = cities_fset.features

# select United States features
us_features = [f for f in cities_features if f.attributes['CNTRY_NAME']=='United States']
us_features[0]

{"geometry": {"x": -9444234.6816, "y": 4607859.9877, "spatialReference": {"wkid": 102100, "latestWkid": 3857}}, "attributes": {"OBJECTID_1": 2, "Id": 0, "ADMIN_NAME": "Kentucky", "CITY_NAME": "Frankfort", "CNTRY_NAME": "United States", "FIPS_CNTRY": "US", "GMI_ADMIN": "USA-KEN", "LABEL_FLAG": 0, "ObjectID": 500, "POP": 16315, "POP_CLASS": "Less than 50,000", "POP_RANK": 7, "PORT_ID": 0, "STATUS": "Provincial capital", "GlobalID": "68460c29-1db8-4bee-b6cc-495f560810a5", "SHAPE": {"x": -9444234.6816, "y": 4607859.9877, "spatialReference": {"wkid": 102100, "latestWkid": 3857}}}}

In [11]:
fs_edits = []

for feature in us_features:
    us_edit = feature
    us_edit.attributes['FIPS_CNTRY'] = 'USA'
    fs_edits.append(us_edit)
    
fs_edits[0]

{"geometry": {"x": -9444234.6816, "y": 4607859.9877, "spatialReference": {"wkid": 102100, "latestWkid": 3857}}, "attributes": {"OBJECTID_1": 2, "Id": 0, "ADMIN_NAME": "Kentucky", "CITY_NAME": "Frankfort", "CNTRY_NAME": "United States", "FIPS_CNTRY": "USA", "GMI_ADMIN": "USA-KEN", "LABEL_FLAG": 0, "ObjectID": 500, "POP": 16315, "POP_CLASS": "Less than 50,000", "POP_RANK": 7, "PORT_ID": 0, "STATUS": "Provincial capital", "GlobalID": "68460c29-1db8-4bee-b6cc-495f560810a5", "SHAPE": {"x": -9444234.6816, "y": 4607859.9877, "spatialReference": {"wkid": 102100, "latestWkid": 3857}}}}

In [14]:
update_result = cities.edit_features(updates=fs_edits)
update_result

{'addResults': [],
 'updateResults': [{'objectId': 2,
   'uniqueId': 2,
   'globalId': '68460c29-1db8-4bee-b6cc-495f560810a5',
   'success': True},
  {'objectId': 3,
   'uniqueId': 3,
   'globalId': '48045d62-dfd8-48d2-a152-543c4de039a1',
   'success': True},
  {'objectId': 6,
   'uniqueId': 6,
   'globalId': '88e56670-43f3-42a7-96df-f1ee41df3440',
   'success': True},
  {'objectId': 8,
   'uniqueId': 8,
   'globalId': '60a42435-c45b-4a3d-a478-ac077cdba7b1',
   'success': True},
  {'objectId': 10,
   'uniqueId': 10,
   'globalId': '4220e910-3faf-46c8-bad4-5d4d850654d8',
   'success': True},
  {'objectId': 12,
   'uniqueId': 12,
   'globalId': '479f45e0-674c-43ab-9394-252495e822a0',
   'success': True},
  {'objectId': 14,
   'uniqueId': 14,
   'globalId': 'f06ab2f0-ca91-4dd5-8867-bacef45c5762',
   'success': True},
  {'objectId': 16,
   'uniqueId': 16,
   'globalId': '4bdbf081-0b25-44ed-9a3c-906e0521b251',
   'success': True},
  {'objectId': 17,
   'uniqueId': 17,
   'globalId': '5bc6c9

### Edit the WebHook

In [13]:
# Edit the WebHook, only define the parameters you want to edit
# change_types being updated to a list of specific even types. This means the webhook will now be triggered when
# features are edited or updated
webhook.edit(
    name=None,
    change_types=[WebHookEvents.FEATURESEDITED, WebHookEvents.FEATURESUPDATED],
    hook_url=None,
    signature_key=None,
    active=None,
    schedule_info=None,
    payload_format=None,
)
webhook.properties

{
  "name": "UC2023_webhook",
  "owner": "naubry_geosaurus",
  "id": 139,
  "globalId": "f1e47061-5840-4311-96eb-db0100b64bed",
  "tenantId": 25830,
  "serviceId": 26841,
  "active": true,
  "hookUrl": "https://webhook.site/fae5e5b9-b3a5-4dcf-bf26-25c78acbd11b",
  "serviceUrl": "https://services7.arcgis.com/JEwYeAy2cc8qOe3o/arcgis/rest/services/UC_WebHook_Demo/FeatureServer",
  "signatureKey": "",
  "format": "json",
  "serverGen": 968872,
  "contentType": "application/x-www-form-urlencoded",
  "createdTime": 1689112203282,
  "lastUpdatedTime": 1689112232842,
  "changeTypes": [
    "FeaturesEdited",
    "FeaturesUpdated"
  ],
  "scheduleInfo": {
    "name": "",
    "state": "enabled",
    "recurrenceInfo": {
      "frequency": "second",
      "interval": 30
    }
  }
}

#### Enable and Disable hooks

In [ ]:
webhook_manager.disable_hooks()

In [ ]:
webhook_manager.enable_hooks()

#### Clean up

In [ ]:
# Delete all WebHooks in the WebHook service
webhook_manager.delete_all_hooks()

### By utilizing a webhook, you can automate the process of receiving notifications about changes in your ArcGIS Online organization's services. This can help you keep track of activity and perform necessary actions in response.

More Information: https://developers.arcgis.com/python/api-reference/arcgis.features.managers.html#webhookservicemanager